In [4]:
import os
import json
import shutil
import glob
import torch
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
import logging
import pandas as pd

from ultralytics import YOLO

In [ ]:
# Check if the MPS (Metal Performance Shaders) backend is available for Apple Silicon devices
if torch.backends.mps.is_available():
    device = "mps"
# If MPS is not available, check if CUDA (NVIDIA GPUs) is available
elif torch.cuda.is_available():
    device = "cuda"
# If neither MPS nor CUDA is available, fall back to using the CPU
else:
    device = "cpu"
# Print the device being used for training
print(f"🚀 Using device: {device}")

🚀 Using device: mps


In [7]:
#Step 1: Load `meta.json` and Extract Class Mapping
# Load class names from meta.json
META_JSON_FILE = "dataset/DatasetNinja_RDD2022_MIX/meta.json"
with open(META_JSON_FILE, "r") as meta_file:
    meta_data = json.load(meta_file)

class_map = {cls["title"] : idx for idx, cls in enumerate(meta_data["classes"])}
print("Loaded class mapping:", class_map)

Loaded class mapping: {'alligator crack': 0, 'block crack': 1, 'longitudinal crack': 2, 'other corruption': 3, 'pothole': 4, 'repair': 5, 'transverse crack': 6}


In [8]:
class_map = {
    "longitudinal crack" : "D00",
    "transverse crack" : "D10",
    "alligator crack": "D20",
    "block crack": "D30",
    "pothole": "D40",
    "manhole cover": "D50",
    "other corruption": "D60"
}
class_id_map = {v: k for k, v in enumerate(class_map.values())}
print(class_id_map)

{'D00': 0, 'D10': 1, 'D20': 2, 'D30': 3, 'D40': 4, 'D50': 5, 'D60': 6}


In [ ]:
# Step 1: Creating Data_Yaml File for training the model
DATA_YAML_PATH = "data.yaml"
data_yaml_content = f"""
path: ./
train: yolo_dataset/train/images
val: yolo_dataset/val/images
nc: {len(class_map)}
names: {list(class_map.values())}
"""

with open(DATA_YAML_PATH, "w") as f:
    f.write(data_yaml_content)

print("data.yaml created successfully!")

data.yaml created successfully!


In [ ]:
# Step 2: Load the YOLO model with pre-trained weights
model = YOLO("yolov8s.pt") # Using YOLOv8s as the base model
model.to(device)
results = model.train(
    data=DATA_YAML_PATH,
    epochs=50,
    imgsz=640,
    batch=16,
    device=device,
)

print("Training Completed!")

New https://pypi.org/project/ultralytics/8.3.75 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.74 🚀 Python-3.12.1 torch-2.6.0 MPS (Apple M3 Pro)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=F

train: Scanning /Users/spgohil/Developer/Projects/ROAD-CONDITION-DETECTION/datasets/yolo_dataset/train/labels... 643 images, 223 backgrounds, 0 corrupt: 100%|██████████| 643/643 [00:00<00:00, 4800.47it/s]

train: New cache created: /Users/spgohil/Developer/Projects/ROAD-CONDITION-DETECTION/datasets/yolo_dataset/train/labels.cache



val: Scanning /Users/spgohil/Developer/Projects/ROAD-CONDITION-DETECTION/datasets/yolo_dataset/val/labels... 161 images, 43 backgrounds, 0 corrupt: 100%|██████████| 161/161 [00:00<00:00, 5571.27it/s]

val: New cache created: /Users/spgohil/Developer/Projects/ROAD-CONDITION-DETECTION/datasets/yolo_dataset/val/labels.cache
Plotting labels to runs/detect/train/labels.jpg... 


...

WARNING ⚠️ NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  67%|██████▋   | 4/6 [02:44<01:18, 39.01s/it]

WARNING ⚠️ NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [04:20<00:00, 43.46s/it]


                   all        161        324      0.251     0.0595     0.0585     0.0254
                   D00         71        134      0.356     0.0597     0.0624     0.0307
                   D10         46         74      0.287     0.0405     0.0575     0.0248
                   D20         39         51      0.214     0.0784     0.0662     0.0324
                   D40         16         30      0.132     0.0333     0.0443     0.0172
                   D60         26         35      0.267     0.0857      0.062     0.0219
Speed: 14.4ms preprocess, 468.1ms inference, 0.0ms loss, 200.9ms postprocess per image
Results saved to runs/detect/train
🎉 Training Completed!


In [ ]:
# Step 3: Save the trained model 
model.save("yolo_road_defects.pt")
# Print a message indicating that training is completed
print("🎉 Training Completed and model saved as 'yolo_road_defects.pt'!")

In [ ]:
# Step 4: Evaluate Model
metrics = model.val()

# print(metrics)

# Print evaluation results
print("Evaluation Results:")
print(f"Precision: {metrics['precision']}")
print(f"Recall: {metrics['recall']}")
print(f"mAP@0.5: {metrics['map50']}")
print(f"mAP@0.5:0.95: {metrics['map']}")

Ultralytics 8.3.74 🚀 Python-3.12.1 torch-2.6.0 MPS (Apple M3 Pro)
Model summary (fused): 168 layers, 11,128,293 parameters, 0 gradients, 28.5 GFLOPs


val: Scanning /Users/spgohil/Developer/Projects/ROAD-CONDITION-DETECTION/datasets/yolo_dataset/val/labels.cache... 161 images, 43 backgrounds, 0 corrupt: 100%|██████████| 161/161 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/11 [00:00<?, ?it/s]

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   9%|▉         | 1/11 [00:57<09:34, 57.42s/it]

WARNING ⚠️ NMS time limit 2.800s exceeded


...

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  82%|████████▏ | 9/11 [04:11<00:41, 20.69s/it]

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [04:24<00:00, 24.08s/it]


                   all        161        324      0.254      0.105      0.094     0.0432
                   D00         71        134       0.35      0.157      0.121     0.0555
                   D10         46         74      0.216     0.0636     0.0614     0.0246
                   D20         39         51      0.282      0.157      0.159     0.0885
                   D40         16         30      0.112     0.0333     0.0453     0.0168
                   D60         26         35      0.311      0.114     0.0834     0.0306
Speed: 127.2ms preprocess, 457.2ms inference, 0.0ms loss, 353.9ms postprocess per image
Results saved to runs/detect/train2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x358a82390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_re